Analysis Enriched Data 
===============

# Import and Set Up

In [1]:
import pandas as pd 
import numpy as np
from src.settings import Settings
import matplotlib.pyplot as plt
import geopandas as gpd
from geopy import Nominatim
from collections import Counter
import seaborn as sns
from src.utils.data import *
import uuid

In [2]:
# Settings
settings = Settings(_env_file="paths/.env.dev") 
plt.style.use('fivethirtyeight')
data = pd.read_csv(settings.BASELINE_MDK_ENRICHED_DATA)
data["uuid"] = [uuid.uuid4() for i in range(0, len(data))]

# Descriptive Analysis

In [3]:
#dcat: identifier is unique and can be used for counting
for column in list(data.columns):
    print(f"{column} column unique: {data[column].is_unique}")

dct:title column unique: False
dct:identifier column unique: True
dcat:landingpage column unique: True
dct:description column unique: False
ORG column unique: False
dcat:Distribution.dct:license column unique: False
dcat:theme column unique: False
updated_at column unique: False
added column unique: False
MUSTERDATENSATZ column unique: False
THEMA column unique: False
BEZEICHNUNG column unique: False
uuid column unique: True


# General Descriptive Analysis of whole dataset

In [4]:
#general analysis
print(f"The dataset has {len(data)} entries")
print(f"We have {data.duplicated().any().sum()} duplicates")
print(f"columns: {data.columns}")
print(f"A sample entry looks like this: {data.iloc[4]}")

The dataset has 14195 entries
We have 0 duplicates
columns: Index(['dct:title', 'dct:identifier', 'dcat:landingpage', 'dct:description',
       'ORG', 'dcat:Distribution.dct:license', 'dcat:theme', 'updated_at',
       'added', 'MUSTERDATENSATZ', 'THEMA', 'BEZEICHNUNG', 'uuid'],
      dtype='object')
A sample entry looks like this: dct:title                                            Bebauungsplan Horn 23 Hamburg
dct:identifier                                9198BE2B-ABDD-460C-BAB7-464DAB5D484F
dcat:landingpage                 http://suche.transparenz.hamburg.de/dataset/a9...
dct:description                  Der Bebauungsplan Horn 23 für den Geltungsbere...
ORG                                                                        Hamburg
dcat:Distribution.dct:license          http://dcat-ap.de/def/licenses/dl-by-de/2.0
dcat:theme                                                     Regionen und Städte
updated_at                                                              2021-11-26
ad

In [5]:
thema = len(data["THEMA"].unique())
bezeichnungen = len(data["BEZEICHNUNG"].unique())
musterdatensaetze = len(data["MUSTERDATENSATZ"].unique())

print(f"We have {thema} unique 'Themas', {bezeichnungen} unique 'Bezeichnungen' and {musterdatensaetze} unique 'Musterdatensätze'.")

We have 60 unique 'Themas', 238 unique 'Bezeichnungen' and 285 unique 'Musterdatensätze'.


In [ ]:
list(data["BEZEICHNUNG"].unique())

## DCAT: theme

In [ ]:
# dcat:theme analsis
print('amount of uniquely combined dcat:theme entries', len(data['dcat:theme'].unique())) #2409
print('amount of missing entries in dcat:theme', data['dcat:theme'].isnull().sum())
print('percent of missing dcat:theme', len(data[data['dcat:theme'].notna()])/ len(data))

In [ ]:
#inspecting the ones with a missing theme more: question if there is a default category they fall into
missing_theme = data.copy()
missing_theme['dcat:theme'].fillna('No theme', inplace= True)
missing_theme = missing_theme[missing_theme['dcat:theme'] == 'No theme']

In [ ]:
#Preprocessing dcat:theme
def preprocess_dcat_theme(data:pd.DataFrame)-> List:
    """function filters dcat:theme for unique themes and counts them

    Parameters
    ----------
    data : pd.DataFrame
        dataframe that contains the column "dcat:theme"

    Returns
    -------
    List
       nested and flat lists of all unique dcat themes
    """

    theme_processed = data[['dcat:theme']].copy()
    theme_processed.dropna(axis='rows', inplace=True)

    theme_list = [] #for coocurrence
    for row in theme_processed['dcat:theme']:
        row_list = row.split(",")
        cleaned = []
        for entry in row_list:
            clean_entry = entry.strip()
            cleaned.append(clean_entry)
        theme_list.append(cleaned)


    theme_list_flat = [item for sublist in theme_list for item in sublist]

    #unique_dcat_themes =  list(dict.fromkeys(theme_list_flat)) #for coocurrence
    return theme_list, theme_list_flat


theme_list, theme_list_flat = preprocess_dcat_theme(data)
#detected short 
unique_dcat_themes = ['Bildung', 'Bevölkerung und Gesellschaft', 'Regierung und öffentlicher Sektor', 'Justiz',  'Verkehr', 'Landwirtschaft',  'Wirtschaft und Finanzen', 'Umwelt', 'Regionen und Städte', 'Energie', 'Gesundheit', 'Wissenschaft und Technologie', 'Internationale Themen']

#long formats of the themes
written_themes = ['Bildung, Kultur und Sport', 'Bevölkerung und Gesellschaft', 'Regierung und öffentlicher Sektor', 'Justiz, Rechtssystem und öffentliche Sicherheit', 'Verkehr', 'Landwirtschaft, Fischerei, Forstwirtschaft und Nahrungsmittel', 'Wirtschaft und Finanzen', 'Umwelt', 'Regionen und Städte', 'Energie', 'Gesundheit', 'Wissenschaft und Technologie']

words_cooc_matrix, word_to_id = create_co_occurences_matrix(unique_dcat_themes, theme_list)
normal_matrix = words_cooc_matrix.toarray() #transform to plot

ax = plt.axes()
ax.set_title('Coocurrence of dcat:themes')
sns.set(font_scale=0.8)
sns.heatmap(normal_matrix, xticklabels = written_themes, yticklabels= written_themes, ax = ax, annot=True, annot_kws={"fontsize":8},fmt='g')

In [ ]:
x_labels = ['Regierung und öffentlicher Sektor', 'Regionen und Städte', 'Bevölkerung und Gesellschaft', 'Umwelt',
'Verkehr', 'Wirtschaft und Finanzen', 'Landwirtschaft, Fischerei, Forstwirtschaft und Nahrungsmittel', 
'Justiz, Rechtssystem und öffentliche Sicherheit', 'Gesundheit', 
'Energie', 'Bildung, Kultur und Sport', 'Wissenschaft und Technologie']


#VIS
themes_count = Counter(theme_list_flat)
#deleting 'double' counted labels
del(themes_count['Fischerei'])
del(themes_count['Forstwirtschaft und Nahrungsmittel'])
del(themes_count['Rechtssystem und öffentliche Sicherheit'])
del(themes_count['Kultur und Sport'])
themes_count = themes_count.most_common()

x, height = zip(*themes_count)

# Visualization dcat:theme
fig, ax = plt.subplots()
fig.set_size_inches(6,6)
ax.bar(x, height, color="#FE563E")
ax.set_title("Distribution of dcat:themes (when extracted from list)", fontsize=7)
ax.set_xlabel("DCAT:theme", fontsize=7)
ax.set_xticks(x, labels = x_labels)
plt.xticks(rotation=90, fontsize=7)
plt.yticks(fontsize=7)
plt.show()

In [ ]:
#VIS
themes_count = Counter(theme_list_flat)
themes_count = themes_count.most_common()
x, height = zip(*themes_count)

# Visualization dcat:theme
fig, ax = plt.subplots()
fig.set_size_inches(6,6)
ax.bar(x, height, color="#FE563E")
ax.set_title("Distribution of dcat:themes (when extracted from list)", fontsize=7)
ax.set_xlabel("DCAT:theme", fontsize=7)
plt.xticks(rotation=90, fontsize=7)
plt.yticks(fontsize=7)
plt.show()

# THEMA 

In [ ]:
# Exploration data['THEMA']
#print(data['THEMA'].value_counts())
print('unique themas:', len(data['THEMA'].unique()))
print('amount of missing entries in THEMA', data['THEMA'].isna().sum()) #isnull() also zero


In [ ]:
count_thema = count_categories(df=data, column_name='THEMA')
make_distribution_fig(df_count= count_thema, df=data, counter = 'uuid', variable='THEMA', mode='percentage', title="Verteilung der Datensätze nach Thema")

# BEZEICHNUNG

In [ ]:
# Exploration data['BEZEICHNUNG']
print(data['BEZEICHNUNG'].value_counts())
print('unique Bezeichnungen:', len(data['BEZEICHNUNG'].unique()))

In [ ]:
count_bezeichnung = count_categories(df=data, column_name='BEZEICHNUNG', limit=30)
make_distribution_fig(df_count= count_bezeichnung, df=data, counter = 'uuid', variable='BEZEICHNUNG', mode='percentage', title="Verteilung der Datensätze per Bezeichnung (mehr als 30 Beispiele)")

# MUSTERDATENSATZ

In [ ]:
# Exploration data['BEZEICHNUNG']
print(data['MUSTERDATENSATZ'].value_counts())
print('unique Musterdatensatz:', len(data['MUSTERDATENSATZ'].unique()))

In [ ]:
count_musterdatensatz = count_categories(df=data, column_name='MUSTERDATENSATZ', limit=30)
make_distribution_fig(df_count= count_musterdatensatz, df=data, counter = 'uuid', variable='MUSTERDATENSATZ', mode='percentage', title="Distribution of datasets per Musterdatensatz (more than 30 examples)")

# Timeline

In [ ]:
print(f'The data was updated {len(data["updated_at"].unique())} times' )

data['time_processed'] = pd.to_datetime(data['updated_at'],format='%Y-%m-%d')
print(f"We have data ranging from {data['time_processed'].min().year} until {data['time_processed'].max().year}")

data['time_processed'] = data['time_processed'].dt.to_period('Y')

count_time = count_categories(df=data, column_name='time_processed')
count_time['time_processed'] = count_time['time_processed'].astype(str)
make_distribution_fig(df_count= count_time, df=data, counter = 'uuid', variable='time_processed', mode='distribution')
